In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data = pd.read_csv('drive/My Drive/Colab Notebooks/rec sys/data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
# data = pd.read_csv('data/retail_train.csv')
# data.head(2)

In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [ ]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [ ]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

Random recommendation

In [ ]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [ ]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

result.head(2)

CPU times: user 3.93 s, sys: 42.5 ms, total: 3.97 s
Wall time: 3.98 s


Popularity-based recommendation

In [ ]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [ ]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 138 ms, sys: 0 ns, total: 138 ms
Wall time: 138 ms


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
def calc_log_item_weights(data):
    item_weights = data.groupby(by='item_id')['sales_value'] \
       .sum() \
       .reset_index() \
       .rename(columns={'sales_value':'weight'})
    
    item_weights['weight'] += 1
    item_weights['weight'] = np.log(item_weights['weight'])
    item_weights['weight'] /= item_weights['weight'].sum()
    
    return item_weights

In [ ]:
log_item_weights = calc_log_item_weights(data)

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    
    
    return recs.tolist()

In [ ]:
%%time

result['random_recommendation_log_weight'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))
result.head(2)

CPU times: user 3.11 s, sys: 65.3 ms, total: 3.18 s
Wall time: 3.19 s


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# Получаем список топ 5000 продаваемых идентификаторов товаров
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

# Выбираем в качестве data_train товары из топ 5000
data_train_top5000 = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [ ]:
%%time

items = data_train_top5000.item_id.unique()

result['random_recommendation_top5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 246 ms, sys: 9.09 ms, total: 256 ms
Wall time: 254 ms


In [ ]:

%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train_top5000, n=5)

result['popular_recommendation_top5000'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 44.3 ms, sys: 999 µs, total: 45.3 ms
Wall time: 47.7 ms


In [ ]:
%%time

log_item_weights = calc_log_item_weights(data_train_top5000)

result['random_recommendation_log_weight_top5000'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_item_weights))

CPU times: user 375 ms, sys: 6.66 ms, total: 382 ms
Wall time: 384 ms


In [ ]:
model_names = [m for m in list(result)[2:] ]
k = 5
pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)

,model name,precision@5
0,random_recommendation,0.000784
1,popular_recommendation,0.155240
2,random_recommendation_log_weight,0.000979
3,random_recommendation_top5000,0.005583
4,popular_recommendation_top5000,0.155240
5,random_recommendation_log_weight_top5000,0.006954


In [ ]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
for k in range(1, 10):
    model = ItemItemRecommender(K=k, num_threads=4)                            # k - ближайшие соседи
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)      
    recs = model.recommend(userid=userid_to_id[2],                             
                            user_items=csr_matrix(user_item_matrix).tocsr(),  
                            N=5,                                               # кол-во рекомендаций 
                            filter_already_liked_items=False, 
                            filter_items=None, 
                            recalculate_user=True)
    result[f'itemitem_K={k}'] = result['user_id'].apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

  0%|          | 0/86865 [00:00<?, ?it/s]

In [ ]:
model_names = [m for m in list(result)[2:] ]
k = 5
final_metrics = pd.DataFrame(
    [
        (model_name, result.apply(lambda row: precision_at_k(recommended_list=row[model_name], 
                                                             bought_list=row['actual'], 
                                                             k=k), 
                                                             axis=1).mean())
        for model_name in model_names
    ], 
    columns=['model name', f'precision@{k}']
)
final_metrics.sort_values(by=f'precision@{k}', ascending=False)

,model name,precision@5
12,itemitem_K=7,0.163565
13,itemitem_K=8,0.163565
14,itemitem_K=9,0.163565
1,popular_recommendation,0.155240
4,popular_recommendation_top5000,0.155240
9,itemitem_K=4,0.145739
10,itemitem_K=5,0.145739
11,itemitem_K=6,0.145739
7,itemitem_K=2,0.137414
8,itemitem_K=3,0.128795
